# Stock Options Analysis

Collects stock price data and options chain data, then computes skew, volume ratio,
and out-of-the-money (OTM) activity metrics for each ticker.

**Metrics computed per stock:**
- **Price**: Last close, 20-day realized volatility
- **Skew**: ATM IV, 25-delta put-call IV skew, OTM put IV premium over ATM
- **Volume ratios**: Put/Call volume ratio, Put/Call open interest ratio
- **OTM activity**: OTM put & call volume shares, far-OTM (>10%) put activity

In [ ]:
!pip install yfinance -q

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import warnings
from datetime import datetime, timedelta

warnings.filterwarnings('ignore')

In [ ]:
# ---------- configuration ----------
TICKERS = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META', 'TSLA', 'NVDA', 'JPM', 'SPY', 'QQQ']

# How many of the nearest expiration dates to use (keeps runtime reasonable)
MAX_EXPIRIES = 4

# Moneyness thresholds (as fraction of spot price)
OTM_THRESHOLD = 0.05     # 5% out of the money
FAR_OTM_THRESHOLD = 0.10 # 10% out of the money

In [ ]:
def get_price_data(ticker_obj):
    """
    Fetch recent price history and compute basic stats.

    Parameters
    ----------
    ticker_obj : yf.Ticker

    Returns
    -------
    dict with 'last_close' and 'realized_vol_20d'
    """
    hist = ticker_obj.history(period='3mo')
    if hist.empty:
        return {'last_close': np.nan, 'realized_vol_20d': np.nan}
    last_close = hist['Close'].iloc[-1]
    log_ret = np.log(hist['Close'] / hist['Close'].shift(1)).dropna()
    realized_vol = log_ret.tail(20).std() * np.sqrt(252)
    return {'last_close': last_close, 'realized_vol_20d': realized_vol}

In [ ]:
def collect_options_chain(ticker_obj, max_expiries=MAX_EXPIRIES):
    """
    Collect calls and puts DataFrames for the nearest expiration dates.

    Parameters
    ----------
    ticker_obj : yf.Ticker
    max_expiries : int

    Returns
    -------
    (pd.DataFrame, pd.DataFrame) : combined calls, combined puts
        Each has an extra column 'expiry' with the expiration date string.
    """
    expiries = ticker_obj.options
    if not expiries:
        return pd.DataFrame(), pd.DataFrame()

    all_calls, all_puts = [], []
    for exp in expiries[:max_expiries]:
        chain = ticker_obj.option_chain(exp)
        calls = chain.calls.copy()
        puts = chain.puts.copy()
        calls['expiry'] = exp
        puts['expiry'] = exp
        all_calls.append(calls)
        all_puts.append(puts)

    calls_df = pd.concat(all_calls, ignore_index=True)
    puts_df = pd.concat(all_puts, ignore_index=True)
    return calls_df, puts_df

In [ ]:
def compute_skew_metrics(calls_df, puts_df, spot):
    """
    Compute implied-volatility skew metrics.

    Metrics
    -------
    atm_call_iv : IV of the call strike closest to spot (nearest expiry)
    atm_put_iv  : IV of the put strike closest to spot (nearest expiry)
    skew_25d    : (25-delta put IV proxy) - (25-delta call IV proxy)
                  approximated as IV at 95% strike minus IV at 105% strike
    otm_put_iv_premium : mean IV of 5-10% OTM puts minus ATM put IV

    Parameters
    ----------
    calls_df, puts_df : pd.DataFrame
    spot : float

    Returns
    -------
    dict
    """
    result = {
        'atm_call_iv': np.nan,
        'atm_put_iv': np.nan,
        'skew_25d': np.nan,
        'otm_put_iv_premium': np.nan,
    }
    if calls_df.empty or puts_df.empty or np.isnan(spot):
        return result

    # Use the nearest expiry only for skew
    nearest_exp = sorted(calls_df['expiry'].unique())[0]
    c = calls_df[calls_df['expiry'] == nearest_exp].copy()
    p = puts_df[puts_df['expiry'] == nearest_exp].copy()

    if 'impliedVolatility' not in c.columns:
        return result

    # ATM: strike closest to spot
    c['dist'] = (c['strike'] - spot).abs()
    p['dist'] = (p['strike'] - spot).abs()

    atm_call = c.loc[c['dist'].idxmin()]
    atm_put = p.loc[p['dist'].idxmin()]
    result['atm_call_iv'] = atm_call['impliedVolatility']
    result['atm_put_iv'] = atm_put['impliedVolatility']

    # 25-delta skew proxy: IV at ~95% of spot (put side) vs ~105% of spot (call side)
    target_put_strike = spot * 0.95
    target_call_strike = spot * 1.05
    p['dist25'] = (p['strike'] - target_put_strike).abs()
    c['dist25'] = (c['strike'] - target_call_strike).abs()
    put_25d = p.loc[p['dist25'].idxmin()]
    call_25d = c.loc[c['dist25'].idxmin()]
    if put_25d['impliedVolatility'] > 0 and call_25d['impliedVolatility'] > 0:
        result['skew_25d'] = put_25d['impliedVolatility'] - call_25d['impliedVolatility']

    # OTM put IV premium: mean IV of puts 5-10% OTM minus ATM put IV
    otm_mask = (p['strike'] < spot * (1 - OTM_THRESHOLD)) & (p['strike'] >= spot * (1 - FAR_OTM_THRESHOLD))
    otm_puts = p[otm_mask]
    if not otm_puts.empty and atm_put['impliedVolatility'] > 0:
        otm_iv = otm_puts['impliedVolatility'].mean()
        result['otm_put_iv_premium'] = otm_iv - atm_put['impliedVolatility']

    return result

In [ ]:
def compute_volume_metrics(calls_df, puts_df):
    """
    Compute put/call volume and open-interest ratios.

    Parameters
    ----------
    calls_df, puts_df : pd.DataFrame

    Returns
    -------
    dict with pc_volume_ratio, pc_oi_ratio, total_call_volume, total_put_volume
    """
    result = {
        'total_call_vol': np.nan,
        'total_put_vol': np.nan,
        'pc_volume_ratio': np.nan,
        'total_call_oi': np.nan,
        'total_put_oi': np.nan,
        'pc_oi_ratio': np.nan,
    }
    if calls_df.empty or puts_df.empty:
        return result

    total_call_vol = calls_df['volume'].sum()
    total_put_vol = puts_df['volume'].sum()
    total_call_oi = calls_df['openInterest'].sum()
    total_put_oi = puts_df['openInterest'].sum()

    result['total_call_vol'] = total_call_vol
    result['total_put_vol'] = total_put_vol
    result['pc_volume_ratio'] = total_put_vol / total_call_vol if total_call_vol > 0 else np.nan
    result['total_call_oi'] = total_call_oi
    result['total_put_oi'] = total_put_oi
    result['pc_oi_ratio'] = total_put_oi / total_call_oi if total_call_oi > 0 else np.nan

    return result

In [ ]:
def compute_otm_activity(calls_df, puts_df, spot):
    """
    Measure out-of-the-money options activity.

    OTM calls: strike > spot * (1 + OTM_THRESHOLD)
    OTM puts:  strike < spot * (1 - OTM_THRESHOLD)
    Far OTM:   >10% away from spot

    Parameters
    ----------
    calls_df, puts_df : pd.DataFrame
    spot : float

    Returns
    -------
    dict
    """
    result = {
        'otm_call_vol_pct': np.nan,
        'otm_put_vol_pct': np.nan,
        'otm_call_oi_pct': np.nan,
        'otm_put_oi_pct': np.nan,
        'far_otm_put_vol_pct': np.nan,
        'far_otm_call_vol_pct': np.nan,
        'otm_put_avg_iv': np.nan,
        'otm_call_avg_iv': np.nan,
    }
    if calls_df.empty or puts_df.empty or np.isnan(spot):
        return result

    total_call_vol = calls_df['volume'].sum()
    total_put_vol = puts_df['volume'].sum()
    total_call_oi = calls_df['openInterest'].sum()
    total_put_oi = puts_df['openInterest'].sum()

    # OTM masks
    otm_call_mask = calls_df['strike'] > spot * (1 + OTM_THRESHOLD)
    otm_put_mask = puts_df['strike'] < spot * (1 - OTM_THRESHOLD)
    far_otm_call_mask = calls_df['strike'] > spot * (1 + FAR_OTM_THRESHOLD)
    far_otm_put_mask = puts_df['strike'] < spot * (1 - FAR_OTM_THRESHOLD)

    # Volume percentages
    if total_call_vol > 0:
        result['otm_call_vol_pct'] = calls_df.loc[otm_call_mask, 'volume'].sum() / total_call_vol
        result['far_otm_call_vol_pct'] = calls_df.loc[far_otm_call_mask, 'volume'].sum() / total_call_vol
    if total_put_vol > 0:
        result['otm_put_vol_pct'] = puts_df.loc[otm_put_mask, 'volume'].sum() / total_put_vol
        result['far_otm_put_vol_pct'] = puts_df.loc[far_otm_put_mask, 'volume'].sum() / total_put_vol

    # OI percentages
    if total_call_oi > 0:
        result['otm_call_oi_pct'] = calls_df.loc[otm_call_mask, 'openInterest'].sum() / total_call_oi
    if total_put_oi > 0:
        result['otm_put_oi_pct'] = puts_df.loc[otm_put_mask, 'openInterest'].sum() / total_put_oi

    # Average IV of OTM options
    if 'impliedVolatility' in calls_df.columns:
        otm_calls = calls_df[otm_call_mask]
        otm_puts = puts_df[otm_put_mask]
        if not otm_calls.empty:
            result['otm_call_avg_iv'] = otm_calls['impliedVolatility'].mean()
        if not otm_puts.empty:
            result['otm_put_avg_iv'] = otm_puts['impliedVolatility'].mean()

    return result

In [ ]:
def analyse_stock(symbol):
    """
    Run the full analysis pipeline for a single stock.

    Parameters
    ----------
    symbol : str
        Ticker symbol (e.g. 'AAPL').

    Returns
    -------
    dict
        One row of metrics keyed by column name.
    """
    ticker = yf.Ticker(symbol)

    # Step 1: price data
    price_info = get_price_data(ticker)
    spot = price_info['last_close']

    # Step 2: options chain
    calls_df, puts_df = collect_options_chain(ticker)

    # Step 3: metrics
    skew = compute_skew_metrics(calls_df, puts_df, spot)
    volume = compute_volume_metrics(calls_df, puts_df)
    otm = compute_otm_activity(calls_df, puts_df, spot)

    row = {'ticker': symbol}
    row.update(price_info)
    row.update(skew)
    row.update(volume)
    row.update(otm)
    return row

In [ ]:
# ---------- run analysis for all tickers ----------
rows = []
for sym in TICKERS:
    print(f'Analysing {sym} ...')
    try:
        row = analyse_stock(sym)
        rows.append(row)
    except Exception as e:
        print(f'  ERROR for {sym}: {e}')

df = pd.DataFrame(rows).set_index('ticker')
print(f'\nDone. {len(df)} stocks analysed.')

In [ ]:
# ---------- format and display ----------
fmt = {
    'last_close':          '{:.2f}',
    'realized_vol_20d':    '{:.1%}',
    'atm_call_iv':         '{:.1%}',
    'atm_put_iv':          '{:.1%}',
    'skew_25d':            '{:.1%}',
    'otm_put_iv_premium':  '{:.1%}',
    'pc_volume_ratio':     '{:.2f}',
    'pc_oi_ratio':         '{:.2f}',
    'total_call_vol':      '{:,.0f}',
    'total_put_vol':       '{:,.0f}',
    'total_call_oi':       '{:,.0f}',
    'total_put_oi':        '{:,.0f}',
    'otm_call_vol_pct':    '{:.1%}',
    'otm_put_vol_pct':     '{:.1%}',
    'otm_call_oi_pct':     '{:.1%}',
    'otm_put_oi_pct':      '{:.1%}',
    'far_otm_put_vol_pct': '{:.1%}',
    'far_otm_call_vol_pct':'{:.1%}',
    'otm_put_avg_iv':      '{:.1%}',
    'otm_call_avg_iv':     '{:.1%}',
}

df.style.format(fmt, na_rep='—')

## Metric Descriptions

| Metric | Description |
|---|---|
| `last_close` | Most recent closing price |
| `realized_vol_20d` | Annualized 20-day realized volatility (log returns) |
| `atm_call_iv` | Implied vol of at-the-money call (nearest expiry) |
| `atm_put_iv` | Implied vol of at-the-money put (nearest expiry) |
| `skew_25d` | 25-delta skew proxy: IV(95% strike put) − IV(105% strike call) |
| `otm_put_iv_premium` | Mean IV of 5–10% OTM puts minus ATM put IV |
| `pc_volume_ratio` | Total put volume / total call volume |
| `pc_oi_ratio` | Total put open interest / total call open interest |
| `otm_call_vol_pct` | Share of call volume that is >5% OTM |
| `otm_put_vol_pct` | Share of put volume that is >5% OTM |
| `otm_call_oi_pct` | Share of call OI that is >5% OTM |
| `otm_put_oi_pct` | Share of put OI that is >5% OTM |
| `far_otm_put_vol_pct` | Share of put volume that is >10% OTM |
| `far_otm_call_vol_pct` | Share of call volume that is >10% OTM |
| `otm_put_avg_iv` | Average IV across all >5% OTM puts |
| `otm_call_avg_iv` | Average IV across all >5% OTM calls |

In [ ]:
# ---------- side-by-side views ----------

print('=== Skew Metrics ===')
skew_cols = ['last_close', 'realized_vol_20d', 'atm_call_iv', 'atm_put_iv', 'skew_25d', 'otm_put_iv_premium']
display(df[skew_cols])

print('\n=== Volume Ratios ===')
vol_cols = ['total_call_vol', 'total_put_vol', 'pc_volume_ratio', 'total_call_oi', 'total_put_oi', 'pc_oi_ratio']
display(df[vol_cols])

print('\n=== OTM Activity ===')
otm_cols = ['otm_call_vol_pct', 'otm_put_vol_pct', 'otm_call_oi_pct', 'otm_put_oi_pct',
            'far_otm_call_vol_pct', 'far_otm_put_vol_pct', 'otm_call_avg_iv', 'otm_put_avg_iv']
display(df[otm_cols])

In [ ]:
# ---------- rank stocks by key signals ----------
signals = pd.DataFrame(index=df.index)
signals['skew_rank'] = df['skew_25d'].rank(ascending=False)          # higher skew = more bearish hedging
signals['pc_vol_rank'] = df['pc_volume_ratio'].rank(ascending=False) # higher P/C ratio = more put activity
signals['otm_put_rank'] = df['otm_put_vol_pct'].rank(ascending=False)
signals['far_otm_put_rank'] = df['far_otm_put_vol_pct'].rank(ascending=False)
signals['composite_rank'] = signals.mean(axis=1).rank()

print('=== Composite Bearish-Signal Ranking (1 = most bearish options activity) ===')
signals.sort_values('composite_rank')